In [1]:
from google.colab import drive
drive.mount('/content/drive')

directory = '/content/drive/MyDrive/Colab Notebooks/Kollektif_Ogrenme/Proje'

Mounted at /content/drive


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [27]:
import pandas as pd
from datasets import Dataset, DatasetDict

#imdb embedding
imdb_train_directory = directory + "/imdb_dataset/train_imdb_5x_embeddings.pkl"
imdb_test_directory = directory + "/imdb_dataset/test_imdb_embeddings.pkl"

#news embedding
news_train_directory = directory + "/news_dataset/train_news_5x_embeddings.pkl"
news_test_directory = directory + "/news_dataset/test_news_embeddings.pkl"

#imdb dataset
original_imdb_train_directory = directory+"/imdb_dataset/train_imdb_5x.csv"
original_imdb_test_directory = directory+"/imdb_dataset/test_imdb.csv"

#news dataset
original_news_train_directory = directory+"/news_dataset/train_news_5x.csv"
original_news_test_directory = directory+"/news_dataset/test_news.csv"

# CSV dosyalarından veriyi pandas DataFrame'lerine aktar
ds_train = pd.read_csv(original_imdb_train_directory)
ds_test = pd.read_csv(original_imdb_test_directory)

# Pandas DataFrame'lerini Hugging Face Dataset nesnelerine dönüştür
train_ds = Dataset.from_pandas(ds_train)
test_ds = Dataset.from_pandas(ds_test)

# Tek bir DatasetDict nesnesinde topla
ds = DatasetDict({
    "train": train_ds,
    "test": test_ds
})

print(ds)

ds_train = pd.read_csv(original_news_train_directory)
ds_test = pd.read_csv(original_news_test_directory)

train_ds = Dataset.from_pandas(ds_train)
test_ds = Dataset.from_pandas(ds_test)

ds2 = DatasetDict({
    "train": train_ds,
    "test": test_ds
})

print(ds2)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})


# IMDB

## Train

In [28]:
import pickle
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#########################################
#  1) Load IMDB Embeddings (2 classes)
#########################################
with open(imdb_train_directory, "rb") as f:
    X_train_imdb = pickle.load(f)

# Suppose the IMDB labels (0=negative, 1=positive) were also saved in separate PKL files
y_train_imdb = ds['train']['label']

#########################################
#  2) MLP Model for IMDB (2 classes)
#     "More comprehensive" means deeper
#     and/or wider hidden layers.
#########################################
mlp_imdb = MLPClassifier(
    hidden_layer_sizes=(512, 256, 128),  # three hidden layers
    activation='relu',
    solver='adam',
    max_iter=20,
    random_state=42
)

print("\n===== Training MLP on IMDB (2 classes) =====")
mlp_imdb.fit(X_train_imdb, y_train_imdb)






===== Training MLP on IMDB (2 classes) =====


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(512, 256, 128), max_iter=20, random_state=42)

## Test

In [29]:
import pickle
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

with open(imdb_test_directory, "rb") as f:
    X_test_imdb = pickle.load(f)

y_test_imdb = ds['test']['label']

# Predictions
y_pred_imdb = mlp_imdb.predict(X_test_imdb)

# Evaluation
acc_imdb = accuracy_score(y_test_imdb, y_pred_imdb)
# For binary classification, average='binary'
prec_imdb = precision_score(y_test_imdb, y_pred_imdb, average='binary')
rec_imdb = recall_score(y_test_imdb, y_pred_imdb, average='binary')
f1_imdb = f1_score(y_test_imdb, y_pred_imdb, average='binary')

print(f"IMDB Test Accuracy: {acc_imdb:.3f}")
print(f"IMDB Precision:    {prec_imdb:.3f}")
print(f"IMDB Recall:       {rec_imdb:.3f}")
print(f"IMDB F1 Score:     {f1_imdb:.3f}")

IMDB Test Accuracy: 0.776
IMDB Precision:    0.774
IMDB Recall:       0.780
IMDB F1 Score:     0.777


In [13]:
from collections import Counter
from sklearn.metrics import accuracy_score

# 500 elemanlık yeni bir tahmin listesi oluşturacağız.
y_pred_ensemble = []

for i in range(500):
    # n farklı indeksten gelen tahminleri alıyoruz
    preds = [
        y_pred_imdb[0 + i],
        y_pred_imdb[500 + i],
        y_pred_imdb[1000 + i],
        y_pred_imdb[1500 + i],
        y_pred_imdb[2000 + i],
        y_pred_imdb[2500 + i]
    ]

    # En çok tekrar eden (çoğunluk) değeri buluyoruz
    most_common_pred = Counter(preds).most_common(1)[0][0]

    # Çoğunluk değerini yeni tahmin listemize ekliyoruz
    y_pred_ensemble.append(most_common_pred)

# İlk 500 elemanı karşılaştırarak doğruluk skorunu hesapla
acc_imdb_ensemble = accuracy_score(y_test_imdb[:500], y_pred_ensemble)
print("İlk 500 elemana ait çoğunluk tahminlerinin doğruluğu:", acc_imdb_ensemble)

İlk 500 elemana ait çoğunluk tahminlerinin doğruluğu: 0.746


# News

## Train

In [30]:
#########################################
#  3) Load News Embeddings (4 classes)
#########################################
with open(news_train_directory, "rb") as f:
    X_train_news = pickle.load(f)


# Suppose the IMDB labels (0=negative, 1=positive) were also saved in separate PKL files
y_train_news = ds2['train']['label']

#########################################
#  4) MLP Model for News (4 classes)
#########################################
mlp_news = MLPClassifier(
    hidden_layer_sizes=(512, 256, 128),  # three hidden layers
    activation='relu',
    solver='adam',
    max_iter=20,
    random_state=42
)

print("\n===== Training MLP on News (4 classes) =====")
mlp_news.fit(X_train_news, y_train_news)




===== Training MLP on News (4 classes) =====


MLPClassifier(hidden_layer_sizes=(512, 256, 128), max_iter=20, random_state=42)

## Test

In [31]:
with open(news_test_directory, "rb") as f:
    X_test_news = pickle.load(f)

y_test_news = ds2['test']['label']

# Predictions
y_pred_news = mlp_news.predict(X_test_news)

# Evaluation
acc_news = accuracy_score(y_test_news, y_pred_news)
# For multi-class, use average='macro' or 'weighted'
prec_news = precision_score(y_test_news, y_pred_news, average='macro')
rec_news = recall_score(y_test_news, y_pred_news, average='macro')
f1_news = f1_score(y_test_news, y_pred_news, average='macro')

print(f"News Test Accuracy: {acc_news:.3f}")
print(f"News Precision:     {prec_news:.3f}")
print(f"News Recall:        {rec_news:.3f}")
print(f"News F1 Score:      {f1_news:.3f}")

News Test Accuracy: 0.872
News Precision:     0.872
News Recall:        0.872
News F1 Score:      0.871


In [16]:
from collections import Counter
from sklearn.metrics import accuracy_score

# 500 elemanlık yeni bir tahmin listesi oluşturacağız.
y_pred_ensemble = []

for i in range(500):
    # 3 farklı indeksten gelen tahminleri alıyoruz
    preds = [
        y_pred_news[i],
        y_pred_news[500 + i],
        y_pred_news[1000 + i],
        y_pred_news[1500 + i],
        y_pred_news[2000 + i],
        y_pred_news[2500 + i]
    ]

    # En çok tekrar eden (çoğunluk) değeri buluyoruz
    most_common_pred = Counter(preds).most_common(1)[0][0]

    # Çoğunluk değerini yeni tahmin listemize ekliyoruz
    y_pred_ensemble.append(most_common_pred)

# İlk 500 elemanı karşılaştırarak doğruluk skorunu hesapla
acc_news_ensemble = accuracy_score(y_test_news[:500], y_pred_ensemble)
print("İlk 500 elemana ait çoğunluk tahminlerinin doğruluğu:", acc_news_ensemble)

İlk 500 elemana ait çoğunluk tahminlerinin doğruluğu: 0.876
